In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

# ----------------------
# 1. Load CSV
# ----------------------
data = pd.read_csv("greenhouse_data.csv")

# ----------------------
# 2. Time features
# ----------------------
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])
data['Hour'] = data['DateTime'].dt.hour
data['Hour_sin'] = np.sin(2 * np.pi * data['Hour'] / 24)
data['Hour_cos'] = np.cos(2 * np.pi * data['Hour'] / 24)

# ----------------------
# 3. Feature Engineering
# ----------------------
data['Temp_Avg'] = data[['Temperature Front', 'Temperature Middle', 'Temperature Back']].mean(axis=1)
data['Hum_Avg'] = data[['Humidity Front', 'Humidity Middle', 'Humidity Back']].mean(axis=1)

# ----------------------
# 4. Lag features
# ----------------------
lag_steps = 10
for lag in range(1, lag_steps + 1):
    data[f'Temp_lag{lag}'] = data['Temp_Avg'].shift(lag)
    data[f'Hum_lag{lag}'] = data['Hum_Avg'].shift(lag)

# Targets (next-step prediction)
data['Target_Temp'] = data['Temp_Avg'].shift(-1)
data['Target_Hum'] = data['Hum_Avg'].shift(-1)

data.dropna(inplace=True)

# ----------------------
# 5. Feature selection
# ----------------------
features = (
    ['Temp_Avg', 'Hum_Avg'] +
    [f'Temp_lag{i}' for i in range(1, lag_steps + 1)] +
    [f'Hum_lag{i}' for i in range(1, lag_steps + 1)] +
    ['Hour_sin', 'Hour_cos']
)

X = data[features].values
y = data[['Target_Temp', 'Target_Hum']].values

# ----------------------
# 6. Chronological train/test split
# ----------------------
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# ----------------------
# 7. Scale inputs
# ----------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------
# 8. Build refined neural network
# ----------------------
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# ----------------------
# 9. Train with early stopping
# ----------------------
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# ----------------------
# 10. Evaluation
# ----------------------
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MSE: {test_loss:.3f}")
print(f"Test RMSE: {np.sqrt(test_loss):.3f}")
print(f"Test MAE: {test_mae:.3f}")

# ----------------------
# 11. Convert to TensorFlow Lite
# ----------------------
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("greenhouse_model_lag10.tflite", "wb") as f:
    f.write(tflite_model)

# ----------------------
# 12. Save scaler parameters
# ----------------------
scaler_params = pd.DataFrame({
    "feature": features,
    "mean": scaler.mean_,
    "scale": scaler.scale_
})

scaler_params.to_csv("scaler_params.csv", index=False)

d:\Apps\VSCode\Uni\iot\Assignment\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 627.4973 - mae: 15.9839 - val_loss: 66.7945 - val_mae: 6.6282
Epoch 2/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 51.9141 - mae: 5.0509 - val_loss: 7.8574 - val_mae: 2.0539
Epoch 3/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 33.0122 - mae: 4.0932 - val_loss: 4.3826 - val_mae: 1.4965
Epoch 4/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 28.2181 - mae: 3.7764 - val_loss: 3.2848 - val_mae: 1.2634
Epoch 5/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 24.7636 - mae: 3.5300 - val_loss: 4.6557 - val_mae: 1.6747
Epoch 6/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 22.9620 - mae: 3.3734 - val_loss: 3.0975 - val_mae: 1.2525
Epoch 7/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 21.8157 - mae: 3.2516 - val_loss: 3.7188 - val_mae: 1.4169
Epoch 8/200
766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 20.4552 - mae: 3.1481 - val_loss: 3.4518 - val_mae: 1.3358
Epoch 9/200
766/766 ━━━━━━━━━━━━━━━━━

INFO:tensorflow:Assets written to: C:\Users\Konrad\AppData\Local\Temp\tmp65n2sokf\assets


Saved artifact at 'C:\Users\Konrad\AppData\Local\Temp\tmp65n2sokf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  1826053517392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1826053519312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1826053520080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1826053520848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1826053520464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1826053010256: TensorSpec(shape=(), dtype=tf.resource, name=None)
